# NHS and GP Administrative Data

Notebook showing how to import administrative data for GP practices and additionally place it into a SQLite database. *(SQLite is a simple file based SQL database that "just works".)*

Data file download URLs identfied via:

- GP Practices: http://systems.digital.nhs.uk/data/ods/datadownloads/gppractice
- High Level Admin: http://systems.digital.nhs.uk/data/ods/datadownloads/haandsa
- Trusts: http://systems.digital.nhs.uk/data/ods/datadownloads/othernhs

In [1]:
#!pip install pandas
#pandas is a python library for working with tabular datasets
#It can be used to import data from CSV files and Excel spreadsheets
import pandas as pd

In [2]:
#SQLite is a file based SQL database included in the Python distribution
import sqlite3
#If you want to build the database from scratch, delete any outstanding copy
#Uncomment and run the following command line (!) command
!rm nhsadmin.sqlite

In [3]:
#Create a connection to the database
con = sqlite3.connect("nhsadmin.sqlite")

In [4]:
#This function helps download and unpack data files
def downloader(typ,url=None):
    !rm downloads/{typ}.zip
    if url is None:
        url='http://systems.digital.nhs.uk/data/ods/datadownloads/data-files/{typ}.zip'.format(typ=typ)
    #Download the data from the HSCIC website
    !wget -P downloads/ {url}
    !rm -r data/{typ}/
    #Unzip the downloaded files into a subdirectory of the data folder, making sure the data dir exists first
    !mkdir -p data
    #The -o flag is overkill - if we hadn't deleted the original folder it would overwirte any similar files
    !unzip -o -d data/{typ} downloads/{typ}.zip

In [5]:
def getData(typ,dates=None):
    downloader(typ)
    if dates is None: df = pd.read_csv('data/{typ}/{typ}.csv'.format(typ=typ),header=None)
    else: df = pd.read_csv('data/{typ}/{typ}.csv'.format(typ=typ),header=None,parse_dates=dates)
    return df

## epraccur

In [6]:
#via http://systems.digital.nhs.uk/data/ods/datadownloads/gppractice
#epraccur is administrative info about GP practices - practice codes, address, etc etc

EPRACCUR='epraccur'
epraccur= getData(EPRACCUR)

--2016-09-25 17:50:42--  http://systems.digital.nhs.uk/data/ods/datadownloads/data-files/epraccur.zip
Resolving systems.digital.nhs.uk... 194.189.27.101
Connecting to systems.digital.nhs.uk|194.189.27.101|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 597146 (583K) [application/zip]
Saving to: 'downloads/epraccur.zip'

epraccur.zip        100%[=====================>] 583.15K  1.26MB/s   in 0.5s   

2016-09-25 17:50:44 (1.26 MB/s) - 'downloads/epraccur.zip' saved [597146/597146]

Archive:  downloads/epraccur.zip
  inflating: data/epraccur/epraccur.csv  
  inflating: data/epraccur/epraccur.pdf  


In [7]:
epraccur.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26],
           dtype='int64')

In [8]:
#Update the column names
#Really, we should do this by loading in the Excel version of the file
#and then extracting the metadata from the spreadsheet to identify the column names
#The following information is extracted from the metadata PDF
cols=['Organisation Code','Name','National Grouping','High Level Health Geography',
      'Address Line 1','Address Line 2','Address Line 3','Address Line 4','Address Line 5','Postcode',
      'Open Date','Close Date','Status Code','Organisation Sub-Type code',
      'Commissioner','Join Provider/Purchaser Date','Left Provider/Purchaser Date','Contact Telephone Number',
      'Null','Null','Null',
      'Amended Record Indicator','Null',
      'Provider/Purchaser','Null','Prescribing Setting','Null']

In [9]:
#Set the column names
epraccur.columns=cols
#Drop the "Available for future use" columns
epraccur.drop('Null', axis=1, inplace=True)
#preview the data
epraccur.head(3)

,Organisation Code,Name,National Grouping,High Level Health Geography,Address Line 1,Address Line 2,Address Line 3,Address Line 4,Address Line 5,Postcode,...,Close Date,Status Code,Organisation Sub-Type code,Commissioner,Join Provider/Purchaser Date,Left Provider/Purchaser Date,Contact Telephone Number,Amended Record Indicator,Provider/Purchaser,Prescribing Setting
0,A81001,THE DENSHAM SURGERY,Y54,Q74,THE HEALTH CENTRE,LAWSON STREET,STOCKTON-ON-TEES,CLEVELAND,NaN,TS18 1HU,...,NaN,A,B,00K,20130401.0,NaN,01642 672351,0,00K,4
1,A81002,QUEENS PARK MEDICAL CENTRE,Y54,Q74,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,NaN,TS18 2AW,...,NaN,A,B,00K,20130401.0,NaN,01642 679681,0,00K,4
2,A81003,VICTORIA MEDICAL PRACTICE,Y54,Q74,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,NaN,TS26 8DB,...,NaN,A,B,00K,20130401.0,NaN,01429 272945,0,00K,4


In [10]:
#Example showing how to filter on Parent Organisation Code
epraccur[epraccur['Commissioner']=='10L'].head(3)

,Organisation Code,Name,National Grouping,High Level Health Geography,Address Line 1,Address Line 2,Address Line 3,Address Line 4,Address Line 5,Postcode,...,Close Date,Status Code,Organisation Sub-Type code,Commissioner,Join Provider/Purchaser Date,Left Provider/Purchaser Date,Contact Telephone Number,Amended Record Indicator,Provider/Purchaser,Prescribing Setting
5255,J84003,VENTNOR MEDICAL CENTRE,Y57,Q70,VENTNOR MEDICAL CENTRE,3 ALBERT STREET,VENTNOR,ISLE OF WIGHT,NaN,PO38 1EZ,...,NaN,A,B,10L,20130401.0,NaN,01983 857288,0,10L,4
5256,J84004,EAST COWES MEDICAL CENTRE,Y57,Q70,EAST COWES MEDICAL CENTRE,CHURCH PATH,EAST COWES,ISLE OF WIGHT,NaN,PO32 6RP,...,NaN,A,B,10L,20130401.0,NaN,01983 284333,0,10L,4
5257,J84005,ESPLANADE SURGERY,Y57,Q70,THE ESPLANADE SURGERY,19 THE ESPLANADE,RYDE,ISLE OF WIGHT,NaN,PO33 2EH,...,NaN,A,B,10L,20130401.0,NaN,01983 618388,0,10L,4


### Storing the Data in a SQLite3 Database
If we store several administrative files in the same database, we can run linked queries over them using SQL.

In [11]:
tmp=epraccur.set_index(['Organisation Code'])
#If the table exists, replace it, under the assumption we are using a more recent version of the data
tmp.to_sql(con=con, name=EPRACCUR,if_exists='replace')

/usr/local/lib/python3.5/site-packages/pandas/core/generic.py:1165: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [12]:
#We can now run a SQL query over the data
orgcode='J84007'
pd.read_sql_query('SELECT * from {typ} where "Organisation Code"="{orgcode}"'.format(typ=EPRACCUR,orgcode=orgcode), con)

,Organisation Code,Name,National Grouping,High Level Health Geography,Address Line 1,Address Line 2,Address Line 3,Address Line 4,Address Line 5,Postcode,...,Close Date,Status Code,Organisation Sub-Type code,Commissioner,Join Provider/Purchaser Date,Left Provider/Purchaser Date,Contact Telephone Number,Amended Record Indicator,Provider/Purchaser,Prescribing Setting
0,J84007,ST.HELENS MEDICAL CENTRE,Y57,Q70,ST.HELENS MEDICAL CENTRE,UPPER GREEN ROAD,ST.HELENS,ISLE OF WIGHT,None,PO33 1UG,...,None,A,B,10L,20130401.0,None,01983 871828,0,10L,4


## Trusts

In [13]:
ETRUST='etrust'

#via http://systems.digital.nhs.uk/data/ods/datadownloads/othernhs
etrust= getData(ETRUST)
etrust.head(2)

--2016-09-25 17:50:45--  http://systems.digital.nhs.uk/data/ods/datadownloads/data-files/etrust.zip
Resolving systems.digital.nhs.uk... 194.189.27.101
Connecting to systems.digital.nhs.uk|194.189.27.101|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 717137 (700K) [application/zip]
Saving to: 'downloads/etrust.zip'

etrust.zip          100%[=====================>] 700.33K  1.39MB/s   in 0.5s   

2016-09-25 17:50:46 (1.39 MB/s) - 'downloads/etrust.zip' saved [717137/717137]

Archive:  downloads/etrust.zip
  inflating: data/etrust/etrust.csv  
  inflating: data/etrust/etrust.pdf  


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,R1A,WORCESTERSHIRE HEALTH AND CARE NHS TRUST,Y55,Q77,ISAAC MADDOX HOUSE,SHRUB HILL INDUSTRIAL ESTATE,NaN,WORCESTER,WORCESTERSHIRE,WR4 9RW,...,NaN,NaN,NaN,NaN,0,NaN,F,NaN,NaN,NaN
1,R1A01,PATHWAYS SUPPORT SERVICES,Y55,Q77,30A TENBY STREET,NaN,NaN,BIRMINGHAM,WEST MIDLANDS,B1 3EE,...,NaN,NaN,NaN,NaN,0,NaN,F,NaN,NaN,NaN


In [14]:
cols=['Organisation Code','Name','National Grouping','High Level Health Geography',
      'Address Line 1','Address Line 2','Address Line 3','Address Line 4','Address Line 5','Postcode',
      'Open Date','Close Date','Null','Null',
      'Null','Null','Null','Contact Telephone Number',
      'Null','Null','Null',
      'Amended Record Indicator','Null',
      'GOR Code','Null','Null','Null']

In [15]:
etrust.columns=cols
etrust.drop('Null', axis=1, inplace=True)

etrust.head(2)

,Organisation Code,Name,National Grouping,High Level Health Geography,Address Line 1,Address Line 2,Address Line 3,Address Line 4,Address Line 5,Postcode,Open Date,Close Date,Contact Telephone Number,Amended Record Indicator,GOR Code
0,R1A,WORCESTERSHIRE HEALTH AND CARE NHS TRUST,Y55,Q77,ISAAC MADDOX HOUSE,SHRUB HILL INDUSTRIAL ESTATE,NaN,WORCESTER,WORCESTERSHIRE,WR4 9RW,20110701,NaN,NaN,0,F
1,R1A01,PATHWAYS SUPPORT SERVICES,Y55,Q77,30A TENBY STREET,NaN,NaN,BIRMINGHAM,WEST MIDLANDS,B1 3EE,20110701,NaN,NaN,0,F


In [16]:
etrust[etrust['Name'].str.lower().str.contains('Wight'.lower())].head(2)

,Organisation Code,Name,National Grouping,High Level Health Geography,Address Line 1,Address Line 2,Address Line 3,Address Line 4,Address Line 5,Postcode,Open Date,Close Date,Contact Telephone Number,Amended Record Indicator,GOR Code
1226,R1F,ISLE OF WIGHT NHS TRUST,Y57,Q70,ST MARY'S HOSPITAL,PARKHURST ROAD,NaN,NEWPORT,ISLE OF WIGHT,PO30 5TG,20120401,NaN,NaN,0,J
1284,R1FHQ,ISLE OF WIGHT NHS - HQ,Y57,Q70,ST MARY'S HOSPITAL,PARKHURST ROAD,NaN,NEWPORT,ISLE OF WIGHT,PO30 5TG,20120401,NaN,NaN,0,J


### Storing the Data in a SQLite3 Database

In [17]:
tmp=etrust.set_index(['Organisation Code'])
#If the table exists, replace it, under the assumption we are using a more recent version of the data
tmp.to_sql(con=con, name=ETRUST,if_exists='replace')

/usr/local/lib/python3.5/site-packages/pandas/core/generic.py:1165: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [18]:
orgcode='R1F'
pd.read_sql_query('SELECT * from {typ} where "Organisation Code"="{orgcode}"'.format(typ=ETRUST,orgcode=orgcode), con)

,Organisation Code,Name,National Grouping,High Level Health Geography,Address Line 1,Address Line 2,Address Line 3,Address Line 4,Address Line 5,Postcode,Open Date,Close Date,Contact Telephone Number,Amended Record Indicator,GOR Code
0,R1F,ISLE OF WIGHT NHS TRUST,Y57,Q70,ST MARY'S HOSPITAL,PARKHURST ROAD,None,NEWPORT,ISLE OF WIGHT,PO30 5TG,20120401,None,None,0,J


## CCGs

In [19]:
#via http://systems.digital.nhs.uk/data/ods/datadownloads/othernhs
ECCG='eccg'
eccg= getData(ECCG)
eccg.head(2)

--2016-09-25 17:50:47--  http://systems.digital.nhs.uk/data/ods/datadownloads/data-files/eccg.zip
Resolving systems.digital.nhs.uk... 194.189.27.101
Connecting to systems.digital.nhs.uk|194.189.27.101|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19824 (19K) [application/zip]
Saving to: 'downloads/eccg.zip'

eccg.zip            100%[=====================>]  19.36K  --.-KB/s   in 0.04s  

2016-09-25 17:50:47 (541 KB/s) - 'downloads/eccg.zip' saved [19824/19824]

Archive:  downloads/eccg.zip
  inflating: data/eccg/eccg.csv      
  inflating: data/eccg/eccg.pdf      


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,00C,NHS DARLINGTON CCG,Y54,Q74,DR PIPER HOUSE,KING STREET,NaN,DARLINGTON,COUNTY DURHAM,DL3 6JL,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,00D,"NHS DURHAM DALES, EASINGTON AND SEDGEFIELD CCG",Y54,Q74,SEDGEFIELD COMMUNITY HOSPITAL,SALTERS LANE,SEDGEFIELD,STOCKTON-ON-TEES,CLEVELAND,TS21 3EE,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [20]:
cols=['Organisation Code','Name','National Grouping','High Level Health Geography',
      'Address Line 1','Address Line 2','Address Line 3','Address Line 4','Address Line 5','Postcode',
      'Open Date','Close Date','Null','Organisation Sub-Type Code',
      'Null','Null','Null','Null',
      'Null','Null','Null',
      'Amended Record Indicator','Null',
      'Null','Null','Null','Null']

In [21]:
eccg.columns=cols
eccg.drop('Null', axis=1, inplace=True)
eccg.head(2)

,Organisation Code,Name,National Grouping,High Level Health Geography,Address Line 1,Address Line 2,Address Line 3,Address Line 4,Address Line 5,Postcode,Open Date,Close Date,Organisation Sub-Type Code,Amended Record Indicator
0,00C,NHS DARLINGTON CCG,Y54,Q74,DR PIPER HOUSE,KING STREET,NaN,DARLINGTON,COUNTY DURHAM,DL3 6JL,20130401,NaN,C,0
1,00D,"NHS DURHAM DALES, EASINGTON AND SEDGEFIELD CCG",Y54,Q74,SEDGEFIELD COMMUNITY HOSPITAL,SALTERS LANE,SEDGEFIELD,STOCKTON-ON-TEES,CLEVELAND,TS21 3EE,20130401,NaN,C,0


In [22]:
eccg[eccg['Name'].str.lower().str.contains('Wight'.lower())]

,Organisation Code,Name,National Grouping,High Level Health Geography,Address Line 1,Address Line 2,Address Line 3,Address Line 4,Address Line 5,Postcode,Open Date,Close Date,Organisation Sub-Type Code,Amended Record Indicator
171,10L,NHS ISLE OF WIGHT CCG,Y57,Q70,SOUTH BLOCK,ST MARY'S HOSPITAL,PARKHURST ROAD,NEWPORT,ISLE OF WIGHT,PO30 5TG,20130401,NaN,C,0


### Storing the Data in a SQLite3 Database

In [23]:
tmp=eccg.set_index(['Organisation Code'])
#If the table exists, replace it, under the assumption we are using a more recent version of the data
tmp.to_sql(con=con, name=ECCG,if_exists='replace')

/usr/local/lib/python3.5/site-packages/pandas/core/generic.py:1165: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [24]:
orgcode='10L'
pd.read_sql_query('SELECT * from {typ} where "Organisation Code"="{orgcode}"'.format(typ=ECCG,orgcode=orgcode), con)

,Organisation Code,Name,National Grouping,High Level Health Geography,Address Line 1,Address Line 2,Address Line 3,Address Line 4,Address Line 5,Postcode,Open Date,Close Date,Organisation Sub-Type Code,Amended Record Indicator
0,10L,NHS ISLE OF WIGHT CCG,Y57,Q70,SOUTH BLOCK,ST MARY'S HOSPITAL,PARKHURST ROAD,NEWPORT,ISLE OF WIGHT,PO30 5TG,20130401,None,C,0


In [25]:
#We can now see the benefit of having data from mulitple source data files in the same database
#For example, we can run queries across joined tables such as finding GP Practices by CCG
ccg='NHS ISLE OF WIGHT CCG'
q='''
SELECT epraccur."Organisation Code" as code, epraccur.Name as Name 
FROM eccg, epraccur 
WHERE eccg.Name="{}" AND eccg."Organisation Code"=epraccur.Commissioner'''

pd.read_sql_query(q.format(ccg), con)

,code,Name
0,J84003,VENTNOR MEDICAL CENTRE
1,J84004,EAST COWES MEDICAL CENTRE
2,J84005,ESPLANADE SURGERY
3,J84007,ST.HELENS MEDICAL CENTRE
4,J84008,ARGYLL HOUSE
5,J84010,SHANKLIN MEDICAL CENTRE
6,J84011,CARISBROOKE HEALTH CENTRE
7,J84012,TOWER HOUSE SURGERY
8,J84013,SANDOWN HEALTH CENTRE
9,J84014,THE DOWER HOUSE


## Practices in a CCG - epcmem

In [26]:
#via http://systems.digital.nhs.uk/data/ods/datadownloads/gppractice
EPCMEM='epcmem'
epcmem=getData(EPCMEM,dates=[3,4])
epcmem.head(2)

--2016-09-25 17:50:48--  http://systems.digital.nhs.uk/data/ods/datadownloads/data-files/epcmem.zip
Resolving systems.digital.nhs.uk... 194.189.27.101
Connecting to systems.digital.nhs.uk|194.189.27.101|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205985 (201K) [application/zip]
Saving to: 'downloads/epcmem.zip'

epcmem.zip          100%[=====================>] 201.16K  1.05MB/s   in 0.2s   

2016-09-25 17:50:48 (1.05 MB/s) - 'downloads/epcmem.zip' saved [205985/205985]

Archive:  downloads/epcmem.zip
  inflating: data/epcmem/epcmem.csv  
  inflating: data/epcmem/epcmem.pdf  


,0,1,2,3,4,5
0,A81001,4QP36,W,1999-04-01,2001-03-31,0
1,A81001,5E1,W,2001-04-01,2013-03-31,0


In [27]:
cols=['Organisation Code','Parent Organisation Code',
'Parent Organisation Type','Join Parent Date','Left Parent Date','Amended Record Indicator']

In [28]:
epcmem.columns=cols
epcmem.head(2)

,Organisation Code,Parent Organisation Code,Parent Organisation Type,Join Parent Date,Left Parent Date,Amended Record Indicator
0,A81001,4QP36,W,1999-04-01,2001-03-31,0
1,A81001,5E1,W,2001-04-01,2013-03-31,0


### Storing the Data in a SQLite3 Database

In [29]:
tmp=epcmem.set_index(['Organisation Code','Parent Organisation Code'])
#If the table exists, replace it, under the assumption we are using a more recent version of the data
tmp.to_sql(con=con, name=EPCMEM,if_exists='replace')

/usr/local/lib/python3.5/site-packages/pandas/core/generic.py:1165: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [30]:
orgcode='A81001'
pd.read_sql_query('SELECT * from {typ} where "Organisation Code"="{orgcode}"'.format(typ=EPCMEM,orgcode=orgcode), con)

,Organisation Code,Parent Organisation Code,Parent Organisation Type,Join Parent Date,Left Parent Date,Amended Record Indicator
0,A81001,00K,W,2013-04-01 00:00:00,None,0
1,A81001,4QP36,W,1999-04-01 00:00:00,2001-03-31 00:00:00,0
2,A81001,5E1,W,2001-04-01 00:00:00,2013-03-31 00:00:00,0


## Practice Members

In [31]:
EPRACMEM='epracmem'
epracmem=getData(EPRACMEM,dates=[3,4])
epracmem.head(2)

--2016-09-25 17:50:49--  http://systems.digital.nhs.uk/data/ods/datadownloads/data-files/epracmem.zip
Resolving systems.digital.nhs.uk... 194.189.27.101
Connecting to systems.digital.nhs.uk|194.189.27.101|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1210110 (1.2M) [application/zip]
Saving to: 'downloads/epracmem.zip'

epracmem.zip        100%[=====================>]   1.15M  1.49MB/s   in 0.8s   

2016-09-25 17:50:50 (1.49 MB/s) - 'downloads/epracmem.zip' saved [1210110/1210110]

Archive:  downloads/epracmem.zip
  inflating: data/epracmem/epracmem.csv  
  inflating: data/epracmem/epracmem.pdf  


,0,1,2,3,4,5
0,G0102005,H81600,P,1974-04-01,1991-04-01,0
1,G0102926,D81001,P,1974-04-01,1991-12-31,0


In [32]:
cols=['Practitioner Code','Parent Organisation Code','Parent Organisation Type','Join Parent Date',
      'Left Parent Date','Amended Record Indicator']

In [33]:
epracmem.columns=cols
epracmem.head(2)

,Practitioner Code,Parent Organisation Code,Parent Organisation Type,Join Parent Date,Left Parent Date,Amended Record Indicator
0,G0102005,H81600,P,1974-04-01,1991-04-01,0
1,G0102926,D81001,P,1974-04-01,1991-12-31,0


In [34]:
epracmem[epracmem['Parent Organisation Code']=='J84020']

,Practitioner Code,Parent Organisation Code,Parent Organisation Type,Join Parent Date,Left Parent Date,Amended Record Indicator
10181,G3335046,J84020,P,1974-04-01,2006-05-17,0
12450,G3370324,J84020,P,1974-04-01,2006-04-01,0
48301,G8337043,J84020,P,2003-07-07,2008-09-30,0
59043,G8549718,J84020,P,2006-03-20,NaT,0
62086,G8637358,J84020,P,2006-05-02,NaT,0
108472,G9508552,J84020,P,1995-08-14,NaT,0
115777,G9710832,J84020,P,1997-11-17,2005-04-30,0


### Storing the Data in a SQLite3 Database

## egdpprac

In [35]:
EGDPPRAC='egdpprac'
egdpprac=getData(EGDPPRAC)
egdpprac.head(2)

--2016-09-25 17:50:51--  http://systems.digital.nhs.uk/data/ods/datadownloads/data-files/egdpprac.zip
Resolving systems.digital.nhs.uk... 194.189.27.101
Connecting to systems.digital.nhs.uk|194.189.27.101|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375667 (367K) [application/zip]
Saving to: 'downloads/egdpprac.zip'

egdpprac.zip        100%[=====================>] 366.86K  1.22MB/s   in 0.3s   

2016-09-25 17:50:51 (1.22 MB/s) - 'downloads/egdpprac.zip' saved [375667/375667]

Archive:  downloads/egdpprac.zip
  inflating: data/egdpprac/egdpprac.csv  
  inflating: data/egdpprac/egdpprac.pdf  


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,V00002,DENTAL SURGERY,Y52,Q37,DENTAL SURGERY,22 MARTYRS AVENUE,CRAWLEY,WEST SUSSEX,NaN,RH11 7RZ,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,V00003,CRABTREE ROAD DENTAL PRACTICE,Y57,Q81,CRABTREE ROAD DENTAL PRACTICE,25 CRABTREE ROAD,CRAWLEY,WEST SUSSEX,NaN,RH11 7HL,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


### Storing the Data in a SQLite3 Database

In [36]:
EGPARC='egparc'
egparc=getData(EGPARC)
egparc.head(2)

--2016-09-25 17:50:52--  http://systems.digital.nhs.uk/data/ods/datadownloads/data-files/egparc.zip
Resolving systems.digital.nhs.uk... 194.189.27.101
Connecting to systems.digital.nhs.uk|194.189.27.101|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 474533 (463K) [application/zip]
Saving to: 'downloads/egparc.zip'

egparc.zip          100%[=====================>] 463.41K  1.28MB/s   in 0.4s   

2016-09-25 17:50:52 (1.28 MB/s) - 'downloads/egparc.zip' saved [474533/474533]

Archive:  downloads/egparc.zip
  inflating: data/egparc/egparc.csv  
  inflating: data/egparc/egparc.pdf  


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,G0107275,ANGEL AM,Y10,QAJ,NIGHTINGALE HOUSE,105 NIGHTINGALE LANE,BALHAM,LONDON,NaN,SW12 8NB,...,0181 6733495,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,G0108180,ANDERSEN HJ,Y10,QAH,123 EVELINA ROAD,DULWICH,LONDON,NaN,NaN,SE15 3HD,...,071 6393126,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


### Storing the Data in a SQLite3 Database

## egpcur

In [37]:
EGPARC='egparc'
egparc=getData(EGPARC)
egparc.head(2)

--2016-09-25 17:50:53--  http://systems.digital.nhs.uk/data/ods/datadownloads/data-files/egparc.zip
Resolving systems.digital.nhs.uk... 194.189.27.101
Connecting to systems.digital.nhs.uk|194.189.27.101|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 474533 (463K) [application/zip]
Saving to: 'downloads/egparc.zip'

egparc.zip          100%[=====================>] 463.41K  1.30MB/s   in 0.3s   

2016-09-25 17:50:53 (1.30 MB/s) - 'downloads/egparc.zip' saved [474533/474533]

Archive:  downloads/egparc.zip
  inflating: data/egparc/egparc.csv  
  inflating: data/egparc/egparc.pdf  


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,G0107275,ANGEL AM,Y10,QAJ,NIGHTINGALE HOUSE,105 NIGHTINGALE LANE,BALHAM,LONDON,NaN,SW12 8NB,...,0181 6733495,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,G0108180,ANDERSEN HJ,Y10,QAH,123 EVELINA ROAD,DULWICH,LONDON,NaN,NaN,SE15 3HD,...,071 6393126,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


### Storing the Data in a SQLite3 Database

## epracarc

In [38]:
EPRACARC='epracarc'
epracarc=getData(EPRACARC)

--2016-09-25 17:50:54--  http://systems.digital.nhs.uk/data/ods/datadownloads/data-files/epracarc.zip
Resolving systems.digital.nhs.uk... 194.189.27.101
Connecting to systems.digital.nhs.uk|194.189.27.101|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 187761 (183K) [application/zip]
Saving to: 'downloads/epracarc.zip'

epracarc.zip        100%[=====================>] 183.36K  1.06MB/s   in 0.2s   

2016-09-25 17:50:54 (1.06 MB/s) - 'downloads/epracarc.zip' saved [187761/187761]

Archive:  downloads/epracarc.zip
  inflating: data/epracarc/epracarc.csv  
  inflating: data/epracarc/epracarc.pdf  


### Storing the Data in a SQLite3 Database